In [1]:
import sys
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import resnet50
from keras_applications import resnext
import pandas as pd
import numpy as np
import os
import cv2

In [2]:
labels = pd.read_csv('Data/train_label.csv')
labels['Type'].value_counts()
sample = pd.read_csv('Data/sample_submission.csv')
num_train_images = 2520  # we choose 3300 images for this assignment. It works for a machine having 8Gb Ram. You can adjust it if your Ram is different. 
split_point = 2240 # split the data into training data [0:3000] and val data [3000:]
print('num_train_images:', num_train_images)
types = sample.columns[1:]
print('Types:', types)
num_classes = len(types)
print('num_classes:', num_classes)

num_train_images: 2520
Types: Index(['distribute', 'ineq', 'integral', 'limit', 'matrix', 'series', 'sqrt'], dtype='object')
num_classes: 7


In [3]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

img_width = 224

def get_image(filename):
    ########################################################################
    # TODO: Your code here...
    ########################################################################
    original = load_img(filename, target_size=(224,224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    return image_batch[0]

In [4]:
x_train = np.zeros((num_train_images, img_width, img_width, 3), dtype=np.uint8)
y_train = np.zeros((num_train_images, num_classes), dtype=np.uint8)

count = 0

for i in range(num_train_images):
    x_train[i] = get_image('Data/train/%s.png' % labels['id'][i])
    pos_arrays = (types == labels['Type'][i]).nonzero() # recall that types is the array of classes
    pos = pos_arrays[0][0]
    y_train[i][pos] = 1
    count += 1
    if(count % 1000 == 0): print(count, 'images loaded')

1000 images loaded
2000 images loaded


# concatenating with nothing changed

In [5]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import vgg16

#getting the features of all three models

#Resnet50 - last layer of features
resnet50_x_train = resnet50.preprocess_input(x_train.copy())
resnet50_model = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features = resnet50_model.predict(resnet50_x_train, batch_size=64, verbose=1)
print(resnet50_features.shape)

#vgg16
vgg16_x_train = vgg16.preprocess_input(x_train.copy())
vgg16_model = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features = vgg16_model.predict(vgg16_x_train, batch_size=64, verbose=1)
print(vgg16_features.shape)

40/40 [==============================] - 203s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 494s 12s/step
(2520, 7, 7, 512)


In [6]:
concat = np.concatenate([resnet50_features, vgg16_features],axis=3)
print(concat.shape)

(2520, 7, 7, 2560)


In [7]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Activation, BatchNormalization

inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [8]:
from tensorflow.keras.models import Model

model_rv = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d (G  (None, 2560)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2560)              0         
                                                                 
 dense (Dense)               (None, 500)               1280500   
                                                                 
 batch_normalization (BatchN  (None, 500)              2000      
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 500)               0     

In [9]:
model_rv.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [10]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv.fit(concat[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)


Epoch 1/30
18/18 [==============================] - 2s 56ms/step - loss: 1.2962 - accuracy: 0.5415 - val_loss: 0.3294 - val_accuracy: 0.8464
Epoch 2/30
18/18 [==============================] - 1s 47ms/step - loss: 0.6286 - accuracy: 0.7705 - val_loss: 0.0805 - val_accuracy: 0.9750
Epoch 3/30
18/18 [==============================] - 1s 46ms/step - loss: 0.5044 - accuracy: 0.8165 - val_loss: 0.0351 - val_accuracy: 0.9929
Epoch 4/30
18/18 [==============================] - 1s 48ms/step - loss: 0.4291 - accuracy: 0.8496 - val_loss: 0.0393 - val_accuracy: 0.9929
Epoch 5/30
18/18 [==============================] - 1s 50ms/step - loss: 0.3779 - accuracy: 0.8723 - val_loss: 0.0350 - val_accuracy: 0.9929
Epoch 6/30
18/18 [==============================] - 1s 46ms/step - loss: 0.3148 - accuracy: 0.8942 - val_loss: 0.0378 - val_accuracy: 0.9929
Epoch 7/30
18/18 [==============================] - 1s 47ms/step - loss: 0.2985 - accuracy: 0.8960 - val_loss: 0.0347 - val_accuracy: 0.9929
Epoch 8/30
18

In [11]:
model_rv.save('model_rv.h5') 

In [13]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model.predict(image_batch1)
        feature_input2 = vgg16_model.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

        #print('Image Name: ',files,' Prediction: ',types[pos])

In [14]:
y_test = pd.read_excel(r'./Data/test_2_label.xlsx',usecols=[1,1])
y_test = y_test.to_numpy()
y_test = np.squeeze(y_test)
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.775


# concatenating with Activation = Sigmoid

In [15]:
resnet50_x_train_sig = resnet50.preprocess_input(x_train.copy())
resnet50_model_sig = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_sig = resnet50_model_sig.predict(resnet50_x_train_sig, batch_size=64, verbose=1)
print(resnet50_features_sig.shape)

#vgg16
vgg16_x_train_sig = vgg16.preprocess_input(x_train.copy())
vgg16_model_sig = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_sig = vgg16_model_sig.predict(vgg16_x_train_sig, batch_size=64, verbose=1)
print(vgg16_features_sig.shape)

40/40 [==============================] - 215s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 520s 13s/step
(2520, 7, 7, 512)


In [16]:
concat_sig = np.concatenate([resnet50_features_sig, vgg16_features_sig],axis=3)
print(concat_sig.shape)

(2520, 7, 7, 2560)


In [17]:
inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("sigmoid")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [18]:
model_rv_sig = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_sig.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d_1   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 2560)              0         
                                                                 
 dense_2 (Dense)             (None, 500)               1280500   
                                                                 
 batch_normalization_1 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 500)               0   

In [19]:
model_rv_sig.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [20]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_sig.fit(concat_sig[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_sig[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 3s 126ms/step - loss: 1.4664 - accuracy: 0.4643 - val_loss: 1.6209 - val_accuracy: 0.3750
Epoch 2/30
18/18 [==============================] - 1s 54ms/step - loss: 0.8558 - accuracy: 0.6938 - val_loss: 0.2460 - val_accuracy: 0.9286
Epoch 3/30
18/18 [==============================] - 1s 48ms/step - loss: 0.6620 - accuracy: 0.7701 - val_loss: 0.1383 - val_accuracy: 0.9643
Epoch 4/30
18/18 [==============================] - 1s 47ms/step - loss: 0.5709 - accuracy: 0.8018 - val_loss: 0.1008 - val_accuracy: 0.9821
Epoch 5/30
18/18 [==============================] - 1s 51ms/step - loss: 0.5135 - accuracy: 0.8241 - val_loss: 0.1014 - val_accuracy: 0.9821
Epoch 6/30
18/18 [==============================] - 1s 48ms/step - loss: 0.4442 - accuracy: 0.8536 - val_loss: 0.0571 - val_accuracy: 0.9929
Epoch 7/30
18/18 [==============================] - 1s 50ms/step - loss: 0.3955 - accuracy: 0.8679 - val_loss: 0.0658 - val_accuracy: 0.9929
Epoch 8/30
1

In [21]:
model_rv_sig.save('model_rv_sig.h5') 

In [22]:
Test_prediction2 = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_sig.predict(image_batch1)
        feature_input2 = vgg16_model_sig.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_sig.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction2.append(pos)

        #print('Image Name: ',files,' Prediction: ',types[pos])

In [23]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.775


# concatenating Activation = Tanh

In [24]:
#Resnet50 - last layer of features
resnet50_x_train_th = resnet50.preprocess_input(x_train.copy())
resnet50_model_th = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_th = resnet50_model_th.predict(resnet50_x_train_th, batch_size=64, verbose=1)
print(resnet50_features_th.shape)

#vgg16
vgg16_x_train_th = vgg16.preprocess_input(x_train.copy())
vgg16_model_th = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_th = vgg16_model_th.predict(vgg16_x_train_th, batch_size=64, verbose=1)
print(vgg16_features_th.shape)

40/40 [==============================] - 194s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 470s 12s/step
(2520, 7, 7, 512)


In [25]:
concat_th = np.concatenate([resnet50_features_th, vgg16_features_th],axis=3)
print(concat_th.shape)

(2520, 7, 7, 2560)


In [26]:
inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("tanh")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [27]:
model_rv_th = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_th.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d_2   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_4 (Dropout)         (None, 2560)              0         
                                                                 
 dense_4 (Dense)             (None, 500)               1280500   
                                                                 
 batch_normalization_2 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   (None, 500)               0   

In [28]:
model_rv_th.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [29]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_th.fit(concat_th[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_th[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 2s 59ms/step - loss: 1.2043 - accuracy: 0.5750 - val_loss: 0.2068 - val_accuracy: 0.9107
Epoch 2/30
18/18 [==============================] - 1s 49ms/step - loss: 0.5803 - accuracy: 0.8054 - val_loss: 0.0364 - val_accuracy: 0.9964
Epoch 3/30
18/18 [==============================] - 1s 45ms/step - loss: 0.4411 - accuracy: 0.8464 - val_loss: 0.0458 - val_accuracy: 0.9929
Epoch 4/30
18/18 [==============================] - 1s 46ms/step - loss: 0.3539 - accuracy: 0.8835 - val_loss: 0.0304 - val_accuracy: 0.9929
Epoch 5/30
18/18 [==============================] - 1s 46ms/step - loss: 0.3456 - accuracy: 0.8763 - val_loss: 0.0234 - val_accuracy: 0.9929
Epoch 6/30
18/18 [==============================] - 1s 47ms/step - loss: 0.2896 - accuracy: 0.9045 - val_loss: 0.0239 - val_accuracy: 0.9929
Epoch 7/30
18/18 [==============================] - 1s 48ms/step - loss: 0.2788 - accuracy: 0.8982 - val_loss: 0.0344 - val_accuracy: 0.9929
Epoch 8/30
18

In [34]:
model_rv_th.save('model_rv_th.h5') 

In [31]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_th.predict(image_batch1)
        feature_input2 = vgg16_model_th.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_th.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

        #print('Image Name: ',files,' Prediction: ',types[pos])

In [32]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7678571428571429


# concatenating with Activation = softmax

In [35]:
#Resnet50 - last layer of features
resnet50_x_train_s = resnet50.preprocess_input(x_train.copy())
resnet50_model_s = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_s = resnet50_model_s.predict(resnet50_x_train_s, batch_size=64, verbose=1)
print(resnet50_features_s.shape)

#vgg16
vgg16_x_train_s = vgg16.preprocess_input(x_train.copy())
vgg16_model_s = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_s = vgg16_model_s.predict(vgg16_x_train_s, batch_size=64, verbose=1)
print(vgg16_features_s.shape)

40/40 [==============================] - 220s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 509s 13s/step
(2520, 7, 7, 512)


In [36]:
concat_s = np.concatenate([resnet50_features_s, vgg16_features_s],axis=3)
print(concat_s.shape)

(2520, 7, 7, 2560)


In [41]:
inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("softmax")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [42]:
model_rv_s = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_s.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d_4   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_8 (Dropout)         (None, 2560)              0         
                                                                 
 dense_8 (Dense)             (None, 500)               1280500   
                                                                 
 batch_normalization_4 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 500)               0   

In [43]:
model_rv_s.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [44]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_s.fit(concat_s[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_s[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 3s 64ms/step - loss: 1.9338 - accuracy: 0.4009 - val_loss: 1.9518 - val_accuracy: 0.0000e+00
Epoch 2/30
18/18 [==============================] - 1s 57ms/step - loss: 1.9144 - accuracy: 0.6031 - val_loss: 1.9771 - val_accuracy: 0.0000e+00
Epoch 3/30
18/18 [==============================] - 1s 53ms/step - loss: 1.8977 - accuracy: 0.6687 - val_loss: 2.0005 - val_accuracy: 0.0000e+00
Epoch 4/30
18/18 [==============================] - 1s 53ms/step - loss: 1.8807 - accuracy: 0.7201 - val_loss: 2.0150 - val_accuracy: 0.0000e+00
Epoch 4: early stopping


In [45]:
model_rv_s.save('model_rv_s.h5') 

In [46]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_s.predict(image_batch1)
        feature_input2 = vgg16_model_s.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_s.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [47]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.4607142857142857


# Contatenating with Activation = elu

In [6]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import vgg16

resnet50_x_train_elu = resnet50.preprocess_input(x_train.copy())
resnet50_model_elu = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_elu = resnet50_model_elu.predict(resnet50_x_train_elu, batch_size=64, verbose=1)
print(resnet50_features_elu.shape)

#vgg16
vgg16_x_train_elu = vgg16.preprocess_input(x_train.copy())
vgg16_model_elu = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_elu = vgg16_model_elu.predict(vgg16_x_train_elu, batch_size=64, verbose=1)
print(vgg16_features_elu.shape)

40/40 [==============================] - 206s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 538s 13s/step
(2520, 7, 7, 512)


In [7]:
concat_elu = np.concatenate([resnet50_features_elu, vgg16_features_elu],axis=3)
print(concat_elu.shape)

(2520, 7, 7, 2560)


In [9]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Activation, BatchNormalization

inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("elu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [11]:
from tensorflow.keras.models import Model
model_rv_elu = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_elu.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d (G  (None, 2560)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2560)              0         
                                                                 
 dense (Dense)               (None, 500)               1280500   
                                                                 
 batch_normalization (BatchN  (None, 500)              2000      
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 500)               0     

In [12]:
model_rv_elu.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_elu.fit(concat_elu[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_elu[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 4s 82ms/step - loss: 1.3333 - accuracy: 0.5527 - val_loss: 0.4050 - val_accuracy: 0.8107
Epoch 2/30
18/18 [==============================] - 1s 58ms/step - loss: 0.6092 - accuracy: 0.7902 - val_loss: 0.0249 - val_accuracy: 0.9964
Epoch 3/30
18/18 [==============================] - 1s 55ms/step - loss: 0.4412 - accuracy: 0.8482 - val_loss: 0.0175 - val_accuracy: 0.9929
Epoch 4/30
18/18 [==============================] - 1s 53ms/step - loss: 0.3882 - accuracy: 0.8576 - val_loss: 0.0236 - val_accuracy: 0.9929
Epoch 5/30
18/18 [==============================] - 1s 53ms/step - loss: 0.3443 - accuracy: 0.8795 - val_loss: 0.0408 - val_accuracy: 0.9929
Epoch 6/30
18/18 [==============================] - 1s 52ms/step - loss: 0.2955 - accuracy: 0.8987 - val_loss: 0.0303 - val_accuracy: 0.9929
Epoch 6: early stopping


In [15]:
model_rv_elu.save('model_rv_elu.h5') 

In [16]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_elu.predict(image_batch1)
        feature_input2 = vgg16_model_elu.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_elu.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [17]:
y_test = pd.read_excel(r'./Data/test_2_label.xlsx',usecols=[1,1])
y_test = y_test.to_numpy()
y_test = np.squeeze(y_test)
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.775


# concatenatin with Activation = exponential

In [18]:
#Resnet50 - last layer of features
resnet50_x_train_expo = resnet50.preprocess_input(x_train.copy())
resnet50_model_expo = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_expo = resnet50_model_expo.predict(resnet50_x_train_expo, batch_size=64, verbose=1)
print(resnet50_features_expo.shape)

#vgg16
vgg16_x_train_expo = vgg16.preprocess_input(x_train.copy())
vgg16_model_expo = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_expo = vgg16_model_expo.predict(vgg16_x_train_expo, batch_size=64, verbose=1)
print(vgg16_features_expo.shape)

40/40 [==============================] - 219s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 526s 13s/step
(2520, 7, 7, 512)


In [20]:
concat_expo = np.concatenate([resnet50_features_expo, vgg16_features_expo],axis=3)
print(concat_expo.shape)

(2520, 7, 7, 2560)


In [21]:
inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("exponential")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [22]:
model_rv_expo = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_expo.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d_1   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 2560)              0         
                                                                 
 dense_2 (Dense)             (None, 500)               1280500   
                                                                 
 batch_normalization_1 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 500)               0   

In [23]:
model_rv_expo.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [24]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_expo.fit(concat_expo[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_expo[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 3s 76ms/step - loss: 3.4225 - accuracy: 0.4415 - val_loss: 1.7831 - val_accuracy: 0.8357
Epoch 2/30
18/18 [==============================] - 1s 56ms/step - loss: 1.5049 - accuracy: 0.6902 - val_loss: 0.1227 - val_accuracy: 0.9821
Epoch 3/30
18/18 [==============================] - 1s 58ms/step - loss: 1.1931 - accuracy: 0.7353 - val_loss: 0.0566 - val_accuracy: 0.9964
Epoch 4/30
18/18 [==============================] - 1s 51ms/step - loss: 1.0028 - accuracy: 0.7656 - val_loss: 0.0465 - val_accuracy: 0.9929
Epoch 5/30
18/18 [==============================] - 1s 53ms/step - loss: 0.9389 - accuracy: 0.8076 - val_loss: 0.0422 - val_accuracy: 0.9929
Epoch 6/30
18/18 [==============================] - 1s 53ms/step - loss: 0.6966 - accuracy: 0.8179 - val_loss: 0.0534 - val_accuracy: 0.9929
Epoch 7/30
18/18 [==============================] - 1s 54ms/step - loss: 0.6150 - accuracy: 0.8357 - val_loss: 0.0612 - val_accuracy: 0.9929
Epoch 8/30
18

In [25]:
model_rv_expo.save('model_rv_expo.h5') 

In [26]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_expo.predict(image_batch1)
        feature_input2 = vgg16_model_expo.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_expo.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [27]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7607142857142857


# concatenating with Optimizer = SGD

In [28]:
#Resnet50 - last layer of features
resnet50_x_train_sgd = resnet50.preprocess_input(x_train.copy())
resnet50_model_sgd = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_sgd = resnet50_model_sgd.predict(resnet50_x_train_sgd, batch_size=64, verbose=1)
print(resnet50_features_sgd.shape)

#vgg16
vgg16_x_train_sgd = vgg16.preprocess_input(x_train.copy())
vgg16_model_sgd = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_sgd = vgg16_model_sgd.predict(vgg16_x_train_sgd, batch_size=64, verbose=1)
print(vgg16_features_sgd.shape)

40/40 [==============================] - 230s 6s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 454s 11s/step
(2520, 7, 7, 512)


In [30]:
concat_sgd = np.concatenate([resnet50_features_sgd, vgg16_features_sgd],axis=3)
print(concat_sgd.shape)

(2520, 7, 7, 2560)


In [31]:
inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [32]:
model_rv_sgd = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_sgd.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d_2   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_4 (Dropout)         (None, 2560)              0         
                                                                 
 dense_4 (Dense)             (None, 500)               1280500   
                                                                 
 batch_normalization_2 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   (None, 500)               0   

In [33]:
model_rv_sgd.compile(loss='categorical_crossentropy', optimizer="sgd", metrics=['accuracy'])

In [34]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_sgd.fit(concat_sgd[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_sgd[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 3s 55ms/step - loss: 2.0695 - accuracy: 0.2799 - val_loss: 3.9133 - val_accuracy: 0.0000e+00
Epoch 2/30
18/18 [==============================] - 1s 39ms/step - loss: 1.6647 - accuracy: 0.3938 - val_loss: 2.9791 - val_accuracy: 0.0143
Epoch 3/30
18/18 [==============================] - 1s 38ms/step - loss: 1.5576 - accuracy: 0.4263 - val_loss: 2.3822 - val_accuracy: 0.0643
Epoch 4/30
18/18 [==============================] - 1s 39ms/step - loss: 1.3922 - accuracy: 0.4759 - val_loss: 1.8582 - val_accuracy: 0.2393
Epoch 5/30
18/18 [==============================] - 1s 39ms/step - loss: 1.3081 - accuracy: 0.5210 - val_loss: 1.4058 - val_accuracy: 0.4643
Epoch 6/30
18/18 [==============================] - 1s 40ms/step - loss: 1.2659 - accuracy: 0.5330 - val_loss: 1.1260 - val_accuracy: 0.5893
Epoch 7/30
18/18 [==============================] - 1s 40ms/step - loss: 1.2102 - accuracy: 0.5509 - val_loss: 1.0465 - val_accuracy: 0.6571
Epoch 8/3

In [35]:
model_rv_sgd.save('model_rv_sgd.h5') 

In [36]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_sgd.predict(image_batch1)
        feature_input2 = vgg16_model_sgd.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_sgd.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [37]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7464285714285714


# concatenating with Optimizer = RMS*****

In [38]:
#Resnet50 - last layer of features
resnet50_x_train_rms = resnet50.preprocess_input(x_train.copy())
resnet50_model_rms = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_rms = resnet50_model_rms.predict(resnet50_x_train_rms, batch_size=64, verbose=1)
print(resnet50_features_rms.shape)

#vgg16
vgg16_x_train_rms = vgg16.preprocess_input(x_train.copy())
vgg16_model_rms = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_rms = vgg16_model_rms.predict(vgg16_x_train_rms, batch_size=64, verbose=1)
print(vgg16_features_rms.shape)

40/40 [==============================] - 206s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 460s 11s/step
(2520, 7, 7, 512)


In [40]:
concat_rms = np.concatenate([resnet50_features_rms, vgg16_features_rms],axis=3)
print(concat_rms.shape)

(2520, 7, 7, 2560)


In [41]:
inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [42]:
model_rv_rms = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_rms.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d_3   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_6 (Dropout)         (None, 2560)              0         
                                                                 
 dense_6 (Dense)             (None, 500)               1280500   
                                                                 
 batch_normalization_3 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation_3 (Activation)   (None, 500)               0   

In [43]:
model_rv_rms.compile(loss='categorical_crossentropy', optimizer="RMSprop", metrics=['accuracy'])

In [44]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_rms.fit(concat_rms[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_rms[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 3s 65ms/step - loss: 1.1114 - accuracy: 0.6080 - val_loss: 0.0739 - val_accuracy: 0.9821
Epoch 2/30
18/18 [==============================] - 1s 51ms/step - loss: 0.5978 - accuracy: 0.7875 - val_loss: 0.0716 - val_accuracy: 0.9821
Epoch 3/30
18/18 [==============================] - 1s 48ms/step - loss: 0.4948 - accuracy: 0.8277 - val_loss: 0.0607 - val_accuracy: 0.9714
Epoch 4/30
18/18 [==============================] - 1s 49ms/step - loss: 0.3994 - accuracy: 0.8665 - val_loss: 0.0476 - val_accuracy: 0.9929
Epoch 5/30
18/18 [==============================] - 1s 50ms/step - loss: 0.3622 - accuracy: 0.8710 - val_loss: 0.0320 - val_accuracy: 0.9929
Epoch 6/30
18/18 [==============================] - 1s 51ms/step - loss: 0.3075 - accuracy: 0.8978 - val_loss: 0.0463 - val_accuracy: 0.9929
Epoch 7/30
18/18 [==============================] - 1s 51ms/step - loss: 0.2776 - accuracy: 0.9058 - val_loss: 0.0357 - val_accuracy: 0.9929
Epoch 8/30
18

In [45]:
model_rv_rms.save('model_rv_rms.h5') 

In [46]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_rms.predict(image_batch1)
        feature_input2 = vgg16_model_rms.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_rms.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [47]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7607142857142857


# concatenating with Otpimizer = Adadlta

In [48]:
#Resnet50 - last layer of features
resnet50_x_train_ada = resnet50.preprocess_input(x_train.copy())
resnet50_model_ada = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_ada = resnet50_model_ada.predict(resnet50_x_train_ada, batch_size=64, verbose=1)
print(resnet50_features_ada.shape)

#vgg16
vgg16_x_train_ada = vgg16.preprocess_input(x_train.copy())
vgg16_model_ada = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_ada = vgg16_model_ada.predict(vgg16_x_train_ada, batch_size=64, verbose=1)
print(vgg16_features_ada.shape)

40/40 [==============================] - 188s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 440s 11s/step
(2520, 7, 7, 512)


In [49]:
concat_ada = np.concatenate([resnet50_features_ada, vgg16_features_ada],axis=3)
print(concat_ada.shape)

(2520, 7, 7, 2560)


In [50]:
inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [51]:
model_rv_ada = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_ada.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d_4   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_8 (Dropout)         (None, 2560)              0         
                                                                 
 dense_8 (Dense)             (None, 500)               1280500   
                                                                 
 batch_normalization_4 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 500)               0   

In [52]:
model_rv_ada.compile(loss='categorical_crossentropy', optimizer="Adadelta", metrics=['accuracy'])

In [53]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_ada.fit(concat_ada[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_ada[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 2s 65ms/step - loss: 2.6187 - accuracy: 0.1478 - val_loss: 2.0745 - val_accuracy: 0.0107
Epoch 2/30
18/18 [==============================] - 1s 54ms/step - loss: 2.5879 - accuracy: 0.1397 - val_loss: 1.8963 - val_accuracy: 0.0214
Epoch 3/30
18/18 [==============================] - 1s 51ms/step - loss: 2.5925 - accuracy: 0.1455 - val_loss: 1.8058 - val_accuracy: 0.0679
Epoch 4/30
18/18 [==============================] - 1s 50ms/step - loss: 2.5958 - accuracy: 0.1509 - val_loss: 1.7392 - val_accuracy: 0.1286
Epoch 5/30
18/18 [==============================] - 1s 50ms/step - loss: 2.5564 - accuracy: 0.1442 - val_loss: 1.7035 - val_accuracy: 0.1821
Epoch 6/30
18/18 [==============================] - 1s 49ms/step - loss: 2.5579 - accuracy: 0.1536 - val_loss: 1.6783 - val_accuracy: 0.2536
Epoch 7/30
18/18 [==============================] - 1s 49ms/step - loss: 2.5259 - accuracy: 0.1531 - val_loss: 1.6628 - val_accuracy: 0.3107
Epoch 8/30
18

In [54]:
model_rv_ada.save('model_rv_ada.h5') 

In [55]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_ada.predict(image_batch1)
        feature_input2 = vgg16_model_ada.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_ada.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [56]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.26071428571428573


# concatenating with Loss = MeanSquaredError

In [57]:
#Resnet50 - last layer of features
resnet50_x_train_mse = resnet50.preprocess_input(x_train.copy())
resnet50_model_mse = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_mse = resnet50_model_mse.predict(resnet50_x_train_mse, batch_size=64, verbose=1)
print(resnet50_features_mse.shape)

#vgg16
vgg16_x_train_mse = vgg16.preprocess_input(x_train.copy())
vgg16_model_mse = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_mse = vgg16_model_mse.predict(vgg16_x_train_mse, batch_size=64, verbose=1)
print(vgg16_features_mse.shape)

40/40 [==============================] - 201s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 501s 12s/step
(2520, 7, 7, 512)


In [58]:
concat_mse = np.concatenate([resnet50_features_mse, vgg16_features_mse],axis=3)
print(concat_mse.shape)

(2520, 7, 7, 2560)


In [59]:
inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [60]:
model_rv_mse = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_mse.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d_5   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_10 (Dropout)        (None, 2560)              0         
                                                                 
 dense_10 (Dense)            (None, 500)               1280500   
                                                                 
 batch_normalization_5 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation_5 (Activation)   (None, 500)               0   

In [61]:
model_rv_mse.compile(loss='mean_squared_error', optimizer="adam", metrics=['accuracy'])

In [62]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_mse.fit(concat_mse[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_mse[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 2s 55ms/step - loss: 0.0956 - accuracy: 0.4826 - val_loss: 0.1032 - val_accuracy: 0.5286
Epoch 2/30
18/18 [==============================] - 1s 49ms/step - loss: 0.0519 - accuracy: 0.7339 - val_loss: 0.0055 - val_accuracy: 0.9714
Epoch 3/30
18/18 [==============================] - 1s 52ms/step - loss: 0.0408 - accuracy: 0.8036 - val_loss: 0.0063 - val_accuracy: 0.9536
Epoch 4/30
18/18 [==============================] - 1s 61ms/step - loss: 0.0332 - accuracy: 0.8326 - val_loss: 0.0018 - val_accuracy: 0.9929
Epoch 5/30
18/18 [==============================] - 1s 45ms/step - loss: 0.0291 - accuracy: 0.8567 - val_loss: 0.0014 - val_accuracy: 0.9929
Epoch 6/30
18/18 [==============================] - 1s 46ms/step - loss: 0.0256 - accuracy: 0.8826 - val_loss: 0.0016 - val_accuracy: 0.9929
Epoch 7/30
18/18 [==============================] - 1s 43ms/step - loss: 0.0245 - accuracy: 0.8906 - val_loss: 0.0015 - val_accuracy: 0.9929
Epoch 8/30
18

In [63]:
model_rv_mse.save('model_rv_mse.h5') 

In [64]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_mse.predict(image_batch1)
        feature_input2 = vgg16_model_mse.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_mse.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [66]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.775


# concatenating with Loss = MeanAbsoluteError

In [5]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import vgg16

#Resnet50 - last layer of features
resnet50_x_train_mae = resnet50.preprocess_input(x_train.copy())
resnet50_model_mae = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_mae = resnet50_model_mae.predict(resnet50_x_train_mae, batch_size=64, verbose=1)
print(resnet50_features_mae.shape)

#vgg16
vgg16_x_train_mae = vgg16.preprocess_input(x_train.copy())
vgg16_model_mae = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_mae = vgg16_model_mae.predict(vgg16_x_train_mae, batch_size=64, verbose=1)
print(vgg16_features_mae.shape)

40/40 [==============================] - 192s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 450s 11s/step
(2520, 7, 7, 512)


In [6]:
concat_mae = np.concatenate([resnet50_features_mae, vgg16_features_mae],axis=3)
print(concat_mae.shape)

(2520, 7, 7, 2560)


In [7]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Activation, BatchNormalization

inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [8]:
from tensorflow.keras.models import Model
model_rv_mae = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_mae.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d (G  (None, 2560)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2560)              0         
                                                                 
 dense (Dense)               (None, 500)               1280500   
                                                                 
 batch_normalization (BatchN  (None, 500)              2000      
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 500)               0     

In [9]:
model_rv_mae.compile(loss='mean_absolute_error', optimizer="adam", metrics=['accuracy'])

In [10]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_mae.fit(concat_mae[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_mae[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 3s 69ms/step - loss: 0.1616 - accuracy: 0.4705 - val_loss: 0.2672 - val_accuracy: 0.0500
Epoch 2/30
18/18 [==============================] - 1s 56ms/step - loss: 0.0945 - accuracy: 0.7138 - val_loss: 0.0842 - val_accuracy: 0.7321
Epoch 3/30
18/18 [==============================] - 1s 65ms/step - loss: 0.0749 - accuracy: 0.7674 - val_loss: 0.0179 - val_accuracy: 0.9393
Epoch 4/30
18/18 [==============================] - 1s 65ms/step - loss: 0.0605 - accuracy: 0.8174 - val_loss: 0.0204 - val_accuracy: 0.9286
Epoch 5/30
18/18 [==============================] - 1s 63ms/step - loss: 0.0521 - accuracy: 0.8518 - val_loss: 0.0168 - val_accuracy: 0.9464
Epoch 6/30
18/18 [==============================] - 1s 68ms/step - loss: 0.0462 - accuracy: 0.8705 - val_loss: 0.0133 - val_accuracy: 0.9607
Epoch 7/30
18/18 [==============================] - 1s 59ms/step - loss: 0.0432 - accuracy: 0.8777 - val_loss: 0.0059 - val_accuracy: 0.9857
Epoch 8/30
18

In [11]:
model_rv_mae.save('model_rv_mae.h5') 

In [12]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_mae.predict(image_batch1)
        feature_input2 = vgg16_model_mae.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_mae.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)


In [13]:
y_test = pd.read_excel(r'./Data/test_2_label.xlsx',usecols=[1,1])
y_test = y_test.to_numpy()
y_test = np.squeeze(y_test)
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.8


# concatenating with Loss = Hinge

In [14]:
#Resnet50 - last layer of features
resnet50_x_train_hinge = resnet50.preprocess_input(x_train.copy())
resnet50_model_hinge = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_hinge = resnet50_model_hinge.predict(resnet50_x_train_hinge, batch_size=64, verbose=1)
print(resnet50_features_hinge.shape)

#vgg16
vgg16_x_train_hinge = vgg16.preprocess_input(x_train.copy())
vgg16_model_hinge = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_hinge = vgg16_model_hinge.predict(vgg16_x_train_hinge, batch_size=64, verbose=1)
print(vgg16_features_hinge.shape)

40/40 [==============================] - 193s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 476s 12s/step
(2520, 7, 7, 512)


In [15]:
concat_hinge = np.concatenate([resnet50_features_hinge, vgg16_features_hinge],axis=3)
print(concat_hinge.shape)

(2520, 7, 7, 2560)


In [16]:
inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [17]:
model_rv_hinge = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_hinge.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d_1   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_2 (Dropout)         (None, 2560)              0         
                                                                 
 dense_2 (Dense)             (None, 500)               1280500   
                                                                 
 batch_normalization_1 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 500)               0   

In [18]:
model_rv_hinge.compile(loss='hinge', optimizer="adam", metrics=['accuracy'])

In [19]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_hinge.fit(concat_hinge[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_hinge[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 2s 91ms/step - loss: 1.0251 - accuracy: 0.4375 - val_loss: 1.1260 - val_accuracy: 0.0250
Epoch 2/30
18/18 [==============================] - 1s 47ms/step - loss: 0.9561 - accuracy: 0.6996 - val_loss: 0.9784 - val_accuracy: 0.5893
Epoch 3/30
18/18 [==============================] - 1s 44ms/step - loss: 0.9310 - accuracy: 0.7772 - val_loss: 0.9035 - val_accuracy: 0.8250
Epoch 4/30
18/18 [==============================] - 1s 45ms/step - loss: 0.9147 - accuracy: 0.8411 - val_loss: 0.8774 - val_accuracy: 0.9250
Epoch 5/30
18/18 [==============================] - 1s 45ms/step - loss: 0.9095 - accuracy: 0.8473 - val_loss: 0.8699 - val_accuracy: 0.9607
Epoch 6/30
18/18 [==============================] - 1s 46ms/step - loss: 0.9027 - accuracy: 0.8674 - val_loss: 0.8669 - val_accuracy: 0.9643
Epoch 7/30
18/18 [==============================] - 1s 44ms/step - loss: 0.8969 - accuracy: 0.8938 - val_loss: 0.8627 - val_accuracy: 0.9857
Epoch 8/30
18

In [20]:
model_rv_hinge.save('model_rv_hinge.h5') 

In [21]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_hinge.predict(image_batch1)
        feature_input2 = vgg16_model_hinge.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_hinge.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [22]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7785714285714286


# concatenating with Loss = SquaredHinge

In [23]:
#Resnet50 - last layer of features
resnet50_x_train_sh = resnet50.preprocess_input(x_train.copy())
resnet50_model_sh = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features_sh = resnet50_model_sh.predict(resnet50_x_train_sh, batch_size=64, verbose=1)
print(resnet50_features_sh.shape)

#vgg16
vgg16_x_train_sh = vgg16.preprocess_input(x_train.copy())
vgg16_model_sh = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features_sh = vgg16_model_sh.predict(vgg16_x_train_sh, batch_size=64, verbose=1)
print(vgg16_features_sh.shape)

40/40 [==============================] - 189s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 485s 12s/step
(2520, 7, 7, 512)


In [24]:
concat_sh = np.concatenate([resnet50_features_sh, vgg16_features_sh],axis=3)
print(concat_sh.shape)

(2520, 7, 7, 2560)


In [25]:
inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [26]:
model_rv_sh = Model(inputs=inputs, outputs=predictions) # specify what is network input, and what is network output
model_rv_sh.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 7, 7, 2560)]      0         
                                                                 
 global_average_pooling2d_2   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_4 (Dropout)         (None, 2560)              0         
                                                                 
 dense_4 (Dense)             (None, 500)               1280500   
                                                                 
 batch_normalization_2 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   (None, 500)               0   

In [27]:
model_rv_sh.compile(loss='squared_hinge', optimizer="adam", metrics=['accuracy'])

In [28]:
early_stop = EarlyStopping(monitor='val_loss',  patience=3, verbose=1)
# stop if loss does not improve for 3 iterations

model_rv_sh.fit(concat_sh[:split_point], y_train[:split_point], batch_size=128, epochs=30, 
              validation_data=(concat_sh[split_point:], y_train[split_point:]), callbacks=[early_stop], verbose=1)

Epoch 1/30
18/18 [==============================] - 2s 58ms/step - loss: 1.1189 - accuracy: 0.4795 - val_loss: 1.0997 - val_accuracy: 0.5429
Epoch 2/30
18/18 [==============================] - 1s 44ms/step - loss: 1.0053 - accuracy: 0.7272 - val_loss: 0.9344 - val_accuracy: 0.8321
Epoch 3/30
18/18 [==============================] - 1s 43ms/step - loss: 0.9708 - accuracy: 0.8040 - val_loss: 0.8840 - val_accuracy: 0.9607
Epoch 4/30
18/18 [==============================] - 1s 43ms/step - loss: 0.9501 - accuracy: 0.8402 - val_loss: 0.8820 - val_accuracy: 0.9643
Epoch 5/30
18/18 [==============================] - 1s 46ms/step - loss: 0.9361 - accuracy: 0.8728 - val_loss: 0.8644 - val_accuracy: 0.9929
Epoch 6/30
18/18 [==============================] - 1s 44ms/step - loss: 0.9293 - accuracy: 0.8804 - val_loss: 0.8660 - val_accuracy: 0.9929
Epoch 7/30
18/18 [==============================] - 1s 46ms/step - loss: 0.9201 - accuracy: 0.9027 - val_loss: 0.8618 - val_accuracy: 0.9929
Epoch 8/30
18

In [29]:
model_rv_sh.save('model_rv_sh.h5') 

In [30]:
Test_prediction = []
for files in os.listdir('./Data/Test_2'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_2/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model_sh.predict(image_batch1)
        feature_input2 = vgg16_model_sh.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model_rv_sh.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

In [31]:
test_acc = (Test_prediction == y_test).mean()
print('Test accuracy: ', test_acc)

Test accuracy:  0.7678571428571429
